# Chapter 9: Computationally Intensive Methods

**Core Goal:** Use computational power to perform inference when analytical solutions are intractable or distributional assumptions are uncertain.

**Motivation:** Classical statistical inference relies on mathematical theory: closed-form distributions, asymptotic approximations, and strong parametric assumptions. Many real-world problems do not fit this framework—distributions are unknown, parameters are high-dimensional, or exact sampling distributions are mathematically intractable. Modern computational power enables a different approach: simulate, resample, and iterate to approximate distributions empirically. These computationally intensive methods trade mathematical elegance for practical applicability, providing valid inference with minimal assumptions. They represent a paradigm shift from analytical derivation to computational approximation.

In [ ]:
import numpy as np
import scipy.stats as stats

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_theme()

## 9.1 Monte Carlo Methods

**Monte Carlo Method:** Use random sampling to approximate quantities that are difficult or impossible to compute analytically.

**Core Principle:** If we can simulate from a distribution, we can approximate any expectation: $E[g(X)] \approx \frac{1}{N}\sum_{i=1}^N g(X_i)$ where $X_1, ..., X_N$ are independent samples.

**Motivation:** Many statistical quantities involve integrals that have no closed form: expected values, probabilities, variances. Monte Carlo methods replace integration with simulation: draw many samples, compute the quantity for each sample, average the results. By the Law of Large Numbers, this approximation converges to the true value as the number of simulations increases. Monte Carlo is particularly powerful for high-dimensional problems where traditional numerical integration becomes infeasible.

### Example: Estimating π

**Method:** Simulate uniform points in square $[0,1] \times [0,1]$. Fraction inside unit circle ≈ $\pi/4$.

In [ ]:
# Simulate N random points in unit square
N = 100000
x, y = np.random.uniform(0, 1, N), np.random.uniform(0, 1, N)

In [ ]:
# Count points inside quarter-circle: x² + y² ≤ 1
inside_circle = (x**2 + y**2) <= 1
pi_estimate = 4 * np.mean(inside_circle)
print(f"π estimate from {N} simulations: {pi_estimate:.5f}")
print(f"True π: {np.pi:.5f}, Error: {abs(pi_estimate - np.pi):.5f}")

### Monte Carlo Integration

**Problem:** Compute $\int_a^b h(x)dx$ when $h$ has no closed-form antiderivative.

**Solution:** If $X \sim \text{Uniform}(a, b)$, then $\int_a^b h(x)dx = (b-a)E[h(X)] \approx (b-a)\frac{1}{N}\sum_{i=1}^N h(X_i)$

In [ ]:
# Estimate integral of exp(x²) from 0 to 1
def h(x):
    return np.exp(x**2)

In [ ]:
# Monte Carlo: Sample uniform on [0,1], evaluate h, average
samples = np.random.uniform(0, 1, 10000)
integral_estimate = 1.0 * np.mean(h(samples))
print(f"Monte Carlo integral estimate: {integral_estimate:.5f}")

**Standard Error of Monte Carlo Estimate:** $\text{SE} = \frac{\sigma}{\sqrt{N}}$ where $\sigma$ is standard deviation of $h(X)$. Accuracy increases with $\sqrt{N}$.

In [ ]:
# Monte Carlo Standard Error: σ/√N
mc_se = np.std(h(samples)) / np.sqrt(len(samples))
print(f"Monte Carlo Standard Error: {mc_se:.5f}")

## 9.2 Permutation Tests

**Permutation Test:** Exact non-parametric test based on permuting group labels to generate the null distribution.

**Null Hypothesis:** Two samples come from the same distribution (treatment has no effect).

**Procedure:**
1. Compute test statistic $T_{obs}$ from observed data
2. Pool all observations, randomly permute group labels
3. Compute test statistic $T_{perm}$ for permuted data
4. Repeat steps 2-3 many times to build null distribution
5. p-value = proportion of permutations where $|T_{perm}| \geq |T_{obs}|$

**Motivation:** Under the null hypothesis of no treatment effect, group labels are arbitrary—any permutation of labels is equally likely. The permutation test exploits this by generating the exact null distribution through all (or many random) permutations. This provides an exact test without distributional assumptions. It is particularly powerful for small samples where asymptotic approximations may be inaccurate.

In [ ]:
# Two groups: Test if treatment has an effect
np.random.seed(42)
control = stats.norm(50, 10).rvs(15)
treatment = stats.norm(55, 10).rvs(15)

In [ ]:
# Observed test statistic: Difference in means
obs_diff = np.mean(treatment) - np.mean(control)
print(f"Observed difference in means: {obs_diff:.2f}")

In [ ]:
# Permutation test: Randomly shuffle group labels, recompute statistic
combined = np.concatenate([control, treatment])
n_control = len(control)

In [ ]:
# Generate null distribution through permutation
n_permutations = 10000
permuted_diffs = []
for _ in range(n_permutations):
    permuted = np.random.permutation(combined)
    perm_control, perm_treatment = permuted[:n_control], permuted[n_control:]
    permuted_diffs.append(np.mean(perm_treatment) - np.mean(perm_control))

In [ ]:
# p-value = proportion of permutations as extreme as observed
p_value_perm = np.mean(np.abs(permuted_diffs) >= np.abs(obs_diff))
print(f"Permutation test p-value: {p_value_perm:.4f}")

In [ ]:
# Compare with parametric t-test
t_result = stats.ttest_ind(treatment, control)
print(f"t-test p-value: {t_result.pvalue:.4f}")

In [ ]:
plt.hist(permuted_diffs, bins=50, density=True, alpha=0.7, edgecolor='black')
plt.axvline(obs_diff, color='r', linewidth=2, label='Observed difference')
plt.axvline(-obs_diff, color='r', linewidth=2, linestyle='--')
plt.xlabel('Difference in Means'); plt.ylabel('Density')
plt.title('Permutation Distribution Under Null Hypothesis'); plt.legend()

**Advantages of Permutation Tests:**
- Exact: No reliance on asymptotic approximations
- Distribution-free: No assumption of normality
- Flexible: Can use any test statistic (mean difference, median difference, variance ratio, etc.)

**Limitation:** Computationally intensive for large samples (though usually 10000 permutations suffice).

## 9.3 Bootstrap (Advanced Applications)

**Bootstrap Principle:** The sample is to the population as the bootstrap sample is to the sample.

**Motivation:** Bootstrap extends beyond simple Standard Error estimation (Chapter 8). Advanced applications include: bias correction, confidence intervals via bootstrap-t, hypothesis testing, and multivariate inference. The bootstrap is particularly valuable for complex statistics where theoretical distributions are unknown.

### Bootstrap Bias Correction

**Bootstrap Bias Estimate:** $\widehat{\text{Bias}} = \bar{\theta}^* - \hat{\theta}$ where $\bar{\theta}^*$ is mean of bootstrap estimates.

**Bias-Corrected Estimator:** $\hat{\theta}_{corrected} = 2\hat{\theta} - \bar{\theta}^*$

In [ ]:
# Example: Estimate correlation coefficient (can be biased in small samples)
np.random.seed(42)
x = stats.norm(0, 1).rvs(20)
y = 0.5 * x + stats.norm(0, 1).rvs(20)

In [ ]:
# Original estimate: r = corr(x, y)
original_corr = np.corrcoef(x, y)[0, 1]
print(f"Original correlation estimate: {original_corr:.4f}")

In [ ]:
# Bootstrap bias correction: Resample pairs (x, y)
n_bootstrap = 5000
bootstrap_corrs = []
for _ in range(n_bootstrap):
    indices = np.random.choice(len(x), len(x), replace=True)
    bootstrap_corrs.append(np.corrcoef(x[indices], y[indices])[0, 1])

In [ ]:
# Bias = E[r*] - r: Difference between bootstrap mean and original estimate
bootstrap_bias = np.mean(bootstrap_corrs) - original_corr
corrected_corr = 2 * original_corr - np.mean(bootstrap_corrs)
print(f"Bootstrap bias estimate: {bootstrap_bias:.4f}")
print(f"Bias-corrected correlation: {corrected_corr:.4f}")

### Bootstrap-t Confidence Interval

**Method:** Use bootstrap to estimate the distribution of the studentized statistic $T = \frac{\hat{\theta} - \theta}{\widehat{SE}(\hat{\theta})}$.

**Advantage:** Better small-sample performance than percentile method, especially for skewed distributions.

In [ ]:
# Original data and estimate
data = stats.expon(scale=10).rvs(30)
original_mean = np.mean(data)

In [ ]:
# Double bootstrap: For each bootstrap sample, compute SE via nested bootstrap
n_bootstrap = 1000
bootstrap_t_statistics = []
for _ in range(n_bootstrap):
    boot_sample = np.random.choice(data, len(data), replace=True)
    boot_mean = np.mean(boot_sample)
    # Nested bootstrap to estimate SE
    nested_means = [np.mean(np.random.choice(boot_sample, len(boot_sample), replace=True)) for _ in range(100)]
    boot_se = np.std(nested_means)
    if boot_se > 0:
        t_stat = (boot_mean - original_mean) / boot_se
        bootstrap_t_statistics.append(t_stat)

In [ ]:
# Bootstrap-t Confidence Interval: θ̂ - t*₁₋α/₂ × SE, θ̂ - t*α/₂ × SE
original_se = np.std(data) / np.sqrt(len(data))
t_quantiles = np.percentile(bootstrap_t_statistics, [97.5, 2.5])
bootstrap_t_ci = [original_mean - t_quantiles[0] * original_se, 
                  original_mean - t_quantiles[1] * original_se]
print(f"Bootstrap-t 95% Confidence Interval: [{bootstrap_t_ci[0]:.2f}, {bootstrap_t_ci[1]:.2f}]")

## 9.4 Cross-Validation

**Cross-Validation:** Method for assessing model performance by training on one subset of data and testing on another.

**k-Fold Cross-Validation:**
1. Split data into k folds
2. For each fold: train on remaining k-1 folds, test on held-out fold
3. Average performance across all k folds

**Motivation:** Evaluating model performance on the same data used for training leads to overly optimistic assessments. Cross-validation provides an honest estimate of out-of-sample performance by systematically holding out different subsets for testing. It is particularly important for model selection (choosing between competing models) and tuning hyperparameters. Leave-one-out cross-validation (k=n) provides nearly unbiased performance estimates but is computationally expensive.

In [ ]:
# Generate data: y = x + x² + noise
np.random.seed(42)
x = np.linspace(-3, 3, 50)
y = x + x**2 + np.random.normal(0, 2, 50)

In [ ]:
# Compare linear vs quadratic model using 5-fold cross-validation
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

In [ ]:
# Linear model: y ~ x
linear_model = LinearRegression()
linear_scores = cross_val_score(linear_model, x.reshape(-1, 1), y, 
                                cv=5, scoring='neg_mean_squared_error')
print(f"Linear model CV Mean Squared Error: {-linear_scores.mean():.2f}")

In [ ]:
# Quadratic model: y ~ x + x²
quadratic_model = make_pipeline(PolynomialFeatures(2), LinearRegression())
quadratic_scores = cross_val_score(quadratic_model, x.reshape(-1, 1), y,
                                  cv=5, scoring='neg_mean_squared_error')
print(f"Quadratic model CV Mean Squared Error: {-quadratic_scores.mean():.2f}")

**Result:** Cross-validation correctly identifies that quadratic model fits better (lower Mean Squared Error), as the true relationship includes x².

### Leave-One-Out Cross-Validation

**Leave-One-Out:** Special case where k = n. Train on n-1 observations, test on the single held-out observation.

**Advantage:** Nearly unbiased estimate of prediction error.

**Disadvantage:** Computationally expensive (requires n model fits).

In [ ]:
# Leave-One-Out Cross-Validation: k = n
from sklearn.model_selection import LeaveOneOut
loo_scores = cross_val_score(quadratic_model, x.reshape(-1, 1), y,
                             cv=LeaveOneOut(), scoring='neg_mean_squared_error')
print(f"Leave-One-Out CV Mean Squared Error: {-loo_scores.mean():.2f}")

## 9.5 Jackknife

**Jackknife:** Resampling method that systematically leaves out one observation at a time.

**Jackknife Estimate of Standard Error:** $\widehat{SE}_{jack} = \sqrt{\frac{n-1}{n}\sum_{i=1}^n(\hat{\theta}_{(-i)} - \bar{\theta}_{(\cdot)})^2}$

where $\hat{\theta}_{(-i)}$ is estimate with $i$-th observation removed, $\bar{\theta}_{(\cdot)}$ is average of jackknife estimates.

**Motivation:** The jackknife predates the bootstrap and provides similar functionality with less computation. By systematically removing each observation and recomputing the statistic, it assesses how sensitive the estimate is to individual observations. This sensitivity translates to an estimate of variability. While bootstrap is more general and accurate, jackknife is simpler and often adequate.

In [ ]:
# Jackknife Standard Error for median
data = stats.norm(50, 10).rvs(30)
original_median = np.median(data)

In [ ]:
# Compute θ̂₍₋ᵢ₎: Estimate with each observation removed
jackknife_estimates = []
for i in range(len(data)):
    jackknife_sample = np.delete(data, i)
    jackknife_estimates.append(np.median(jackknife_sample))

In [ ]:
# Jackknife Standard Error: √[(n-1)/n × Σ(θ̂₍₋ᵢ₎ - θ̄₍·₎)²]
n = len(data)
theta_bar = np.mean(jackknife_estimates)
jackknife_se = np.sqrt((n-1)/n * np.sum((np.array(jackknife_estimates) - theta_bar)**2))
print(f"Jackknife Standard Error of median: {jackknife_se:.2f}")

In [ ]:
# Compare with bootstrap Standard Error
bootstrap_medians = [np.median(np.random.choice(data, len(data), replace=True)) for _ in range(5000)]
bootstrap_se = np.std(bootstrap_medians)
print(f"Bootstrap Standard Error of median: {bootstrap_se:.2f}")

**Comparison:** Jackknife and bootstrap often give similar Standard Error estimates. Bootstrap is more flexible (works for any statistic) and more accurate for complex estimators.

## 9.6 Markov Chain Monte Carlo

**Markov Chain Monte Carlo:** Methods for sampling from complex distributions by constructing a Markov chain whose stationary distribution is the target distribution.

**Problem:** Want to sample from $p(\theta | \text{data})$, but direct sampling is impossible (distribution is known only up to a constant, or is very high-dimensional).

**Solution:** Construct a Markov chain that "wanders" through the parameter space, visiting regions in proportion to their probability under the target distribution.

**Motivation:** Many posterior distributions in Bayesian inference, likelihood functions in complex models, and high-dimensional distributions cannot be sampled directly. Markov Chain Monte Carlo provides a general-purpose sampling algorithm that requires only the ability to evaluate the target density up to a proportionality constant. Once we have samples from the distribution, we can compute any expectation or quantile. This enables Bayesian inference, complex model fitting, and high-dimensional integration that would be impossible analytically.

### Metropolis-Hastings Algorithm

**Algorithm:**
1. Start at initial value $\theta^{(0)}$
2. Propose new value: $\theta^* \sim q(\theta^* | \theta^{(t)})$
3. Compute acceptance probability: $\alpha = \min\left(1, \frac{p(\theta^*)q(\theta^{(t)}|\theta^*)}{p(\theta^{(t)})q(\theta^*|\theta^{(t)})}\right)$
4. With probability $\alpha$: accept $\theta^{(t+1)} = \theta^*$, otherwise $\theta^{(t+1)} = \theta^{(t)}$
5. Repeat steps 2-4

**Key Property:** After sufficient iterations (burn-in), samples approximate the target distribution $p(\theta)$.

In [ ]:
# Example: Sample from mixture of normals using Metropolis-Hastings
# Target: 0.3×N(0,1) + 0.7×N(5,1)
def target_density(x):
    return 0.3 * stats.norm(0, 1).pdf(x) + 0.7 * stats.norm(5, 1).pdf(x)

In [ ]:
# Metropolis-Hastings with Normal proposal: q(θ*|θ) = N(θ, σ²)
def metropolis_hastings(target, n_iterations, proposal_sd=1.0):
    samples = [0]  # Start at θ = 0
    for _ in range(n_iterations):
        current = samples[-1]
        proposed = current + np.random.normal(0, proposal_sd)
        # Acceptance probability: α = min(1, p(θ*)/p(θ))
        acceptance_prob = min(1, target(proposed) / target(current))
        if np.random.uniform() < acceptance_prob:
            samples.append(proposed)
        else:
            samples.append(current)
    return np.array(samples)

In [ ]:
# Run Markov Chain Monte Carlo
np.random.seed(42)
samples = metropolis_hastings(target_density, n_iterations=10000)
print(f"Acceptance rate: {np.mean(np.diff(samples) != 0):.3f}")

In [ ]:
# Discard burn-in (first 1000 samples)
burn_in = 1000
samples_after_burnin = samples[burn_in:]

In [ ]:
# Compare Markov Chain Monte Carlo samples with true distribution
x_grid = np.linspace(-4, 8, 200)
plt.hist(samples_after_burnin, bins=50, density=True, alpha=0.7, label='Markov Chain Monte Carlo samples', edgecolor='black')
plt.plot(x_grid, [target_density(x) for x in x_grid], 'r-', linewidth=2, label='True density')
plt.xlabel('θ'); plt.ylabel('Density'); plt.title('Metropolis-Hastings Sampling')
plt.legend()

**Diagnostics:**
- **Trace plot:** Plot $\theta^{(t)}$ versus $t$ to check convergence and mixing
- **Autocorrelation:** Check correlation between $\theta^{(t)}$ and $\theta^{(t+k)}$ (should decay rapidly)
- **Multiple chains:** Run several chains from different starting points; should converge to same distribution

In [ ]:
# Trace plot: Shows how chain explores parameter space
plt.plot(samples[:2000])
plt.xlabel('Iteration'); plt.ylabel('θ'); plt.title('Trace Plot (First 2000 iterations)')
plt.axhline(0, color='gray', linestyle='--', alpha=0.5)
plt.axhline(5, color='gray', linestyle='--', alpha=0.5)

## 9.7 Practical Considerations

**Choosing Number of Simulations:**
- **Monte Carlo Standard Error:** $\text{SE} \propto 1/\sqrt{N}$
- To reduce Standard Error by factor of 10, need 100× more simulations
- Typical choices: 1000 for rough approximation, 10000 for publishable results

**Computational Cost versus Statistical Accuracy:**
- Law of diminishing returns: doubling simulations only improves accuracy by $\sqrt{2}$
- Often better to improve simulation design (variance reduction, importance sampling) than just adding more samples

**Parallel Computing:**
- Bootstrap, permutation tests, and Monte Carlo methods are "embarrassingly parallel"
- Can achieve near-linear speedup with multiple processors

**When to Use Computational Methods:**
- Distribution is unknown or complex
- Analytical solution is intractable
- Small sample size makes asymptotics questionable
- Want distribution-free inference
- High-dimensional problems where traditional methods fail

## Summary: Computational Inference Framework

**Simulation-based methods trade mathematical theory for computational approximation:**

**Monte Carlo Integration:**
- Replace integrals with sample averages
- Accuracy increases with $\sqrt{N}$
- Useful for expectations, probabilities, and high-dimensional integrals

**Permutation Tests:**
- Generate exact null distribution by permuting labels
- Exact and distribution-free
- Flexible: can use any test statistic

**Bootstrap:**
- Resample from observed data to approximate sampling distribution
- Provides Standard Errors, bias correction, confidence intervals
- Works for complex statistics without theory

**Cross-Validation:**
- Assess out-of-sample model performance
- Essential for model selection and hyperparameter tuning
- k-fold balances computation and accuracy

**Jackknife:**
- Systematic leave-one-out resampling
- Simpler than bootstrap but less flexible
- Useful for Standard Error estimation

**Markov Chain Monte Carlo:**
- Sample from complex distributions via Markov chains
- Enables Bayesian inference and high-dimensional integration
- Requires burn-in and diagnostic checking

**Common theme:** When theory is hard, simulate. Modern computational power makes methods once considered impractical now routine.

## Key Takeaways

- **Computation enables inference without strong assumptions:** Monte Carlo, bootstrap, and permutation tests provide valid inference with minimal distributional assumptions, making them more broadly applicable than classical methods.

- **Accuracy improves slowly with simulation size:** Monte Carlo Standard Error decreases as $1/\sqrt{N}$, so quadrupling simulations only doubles accuracy. This limits practical precision but is usually sufficient for statistical inference.

- **Permutation tests are exact and distribution-free:** By generating the null distribution through all permutations of labels, permutation tests provide exact p-values without assuming normality or relying on asymptotics.

- **Bootstrap is remarkably general:** It works for any statistic (mean, median, correlation, regression coefficients) and provides Standard Errors, bias correction, and confidence intervals using only the observed sample.

- **Cross-validation prevents overfitting:** Training and testing on the same data gives overly optimistic performance estimates. Cross-validation provides honest assessment of out-of-sample performance.

- **Markov Chain Monte Carlo enables Bayesian inference:** By constructing Markov chains that sample from posterior distributions, Markov Chain Monte Carlo makes Bayesian inference practical for complex models where analytical solutions are impossible.

- **Computational methods are embarrassingly parallel:** Bootstrap iterations, permutation replicates, and Monte Carlo simulations are independent, allowing near-linear speedup with multiple processors. This makes large-scale computational inference feasible.